In [6]:
import os
import re
import praw
import time
import string
import pyLDAvis
import numpy as np
import pandas as pd
import seaborn as sns
import pyLDAvis.lda_model
from tabulate import tabulate
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [15]:
reddit = praw.Reddit(
    client_id="OJtDOFbkvxW9oodT0qSozw",
    client_secret="6unjzbEWPaQoomY7uRYq0mQIia80IQ",
    user_agent="moon_rover69"
)

combined_data = []

# subreddit_names = ["UkraineRussiaReport", "RussiaUkraineWar", "RussiaUkraineWar2022", "UkrainianConflict"]
subreddit_names = ["UkraineRussiaReport"]
for subreddit_name in subreddit_names:
    subreddit = reddit.subreddit(subreddit_name)

    print("Fetching upto 1000 posts and top 50 comments each...")

    for idx, submission in enumerate(subreddit.new(limit=10)):
        # Add the post itself
        print(f"Fetching comments for post: {idx}: {submission.title}")
        combined_data.append({
            'id': submission.id,
            'type': 'submission',
            'title': submission.title,
            'selftext': submission.selftext,
            'created_utc': submission.created_utc,
            "upvote_ratio": submission.upvote_ratio,
            "upvote": submission.upvote
        })

    # Fetch comments
        submission.comments.replace_more()
        for comment in submission.comments.list()[:50]:
            combined_data.append({
                'id': comment.id,
                'type': 'comment',
                'title': None,
                'selftext': comment.body,
                'created_utc': comment.created_utc,
                # "upvote_ratio": comment.upvote_ratio,
                "upvote": submission.upvote
            })

        if (idx + 1) % 100 == 0:
            print(f"{idx + 1} posts processed...")
            time.sleep(1)

df = pd.DataFrame(combined_data)
file_path = 'reddit_combined_posts_and_comments_with_upvote.csv'
# If the file doesn't exist yet, write header; otherwise, append without header
write_header = not os.path.exists(file_path)
df.to_csv(
   file_path,
    mode='a',
    header=write_header,
    index=False
)

print("✅ Done! Data saved with matching format.")

Fetching upto 1000 posts and top 50 comments each...
Fetching comments for post: 0: RU POV: Fiber-optic drone hit Ukrainian international MaxxPro.
Fetching comments for post: 1: RU POV: FPV drone strikes on Ukrainian vehicles.
Fetching comments for post: 2: UA POV: In Kharkiv, a drunken soldier fired a pneumatic pistol at minors because he was offended that they called him a TCC officer due to his military uniform - TSN
Fetching comments for post: 3: UA POV: The New German Chancellor Merz stated they willl continue to support Ukraine including militarily and at the same time they hope that there will be a chance for a ceasefire and then for peace talks
Fetching comments for post: 4: RU POV Russia’s Special Economic Zones Attracting New Foreign Investors - RUSSIA'S PIVOT TO ASIA
Fetching comments for post: 5: UA POV: In 4 districts of Poltava region, mobile checkpoints will be established to locate AWOL soldiers, each staffed by 6 police officers, 4 servicemen of the TCC, 1 officer from